In [1]:
# Dependencies

# Import Splinter
from splinter import Browser
# Import BeautifulSoup
from bs4 import BeautifulSoup as soup

import requests
import os
import pandas as pd
import re

from webdriver_manager.chrome import ChromeDriverManager

In [4]:
executable_path = {'executable_path': 'C:/bin/chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
# Function to remove tags
def remove_tags(html):
    
    # parse html content
    soup = BeautifulSoup(html, "html.parser")
    
    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()
    
    # Return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)

In [6]:
# base url
url = 'https://www.olympedia.org/athletes/93860'

## site statistics 
site_statistics = 'https://www.olympedia.org/statistics/olympedia'

In [7]:
browser.visit(url)
# Optional delay for loding the page
browser.is_element_present_by_css('table.biodata', wait_time=1)

True

In [8]:
html = browser.html
news_soup = soup(html, "html.parser")

In [9]:
biodata = news_soup.select_one('table.biodata')
bio_column = biodata.findChildren(['th'])
bio_column

[<th>Type</th>,
 <th>Sex</th>,
 <th>Full name</th>,
 <th>Used name</th>,
 <th>Nick/petnames</th>,
 <th>Born</th>,
 <th>Measurements</th>,
 <th>Affiliations</th>,
 <th>NOC</th>]

In [10]:
bio_column = [str(i).replace(r"<th>", '') for i in bio_column]
bio_column = [str(i).replace(r"</th>", '') for i in bio_column]
bio_column

['Type',
 'Sex',
 'Full name',
 'Used name',
 'Nick/petnames',
 'Born',
 'Measurements',
 'Affiliations',
 'NOC']

In [11]:
biodate = news_soup.select_one('table.biodata')
biodate_columns = pd.DataFrame(biodate.findChildren(['th']))
biodate_columns

,0
0,Type
1,Sex
2,Full name
3,Used name
4,Nick/petnames
5,Born
6,Measurements
7,Affiliations
8,NOC


In [13]:
#biodate.findChildren(['td']).replace('\<\/*td\>','',regex=True)

In [15]:
bio_data = biodata.findChildren(['td'])
bio_data = [str(i).replace(r'<td>', '') for i in bio_data]
bio_data = [str(i).replace(r'</td>', '') for i in bio_data]
bio_data = [str(i).replace(r'•', '') for i in bio_data]

bio_data

['Competed in Olympic Games',
 'Male',
 'Michael FredPhelps, II',
 'MichaelPhelps',
 'The Baltimore Bullet',
 '30 June 1985 in <a href="/place_names/4423">Baltimore, Maryland (USA)</a>',
 '193 cm / 91 kg',
 '<a href="/affiliations/6636">NBAC, Baltimore (USA)</a> / <a href="/affiliations/3411">Michigan Wolverines, Ann Arbor (USA)</a>',
 '<img src="/images/flags/USA.png" style="padding-right: 2px; vertical-align: middle"/> <a href="/countries/USA">United States</a>']

In [16]:
bio_data = [str(i).replace(r'<a href="/affiliations/','') for i in bio_data]
bio_data = [str(i).replace(r'<img src="/images/flags/','') for i in bio_data]
bio_data = [str(i).replace(r'.png.+','') for i in bio_data]
bio_data

['Competed in Olympic Games',
 'Male',
 'Michael FredPhelps, II',
 'MichaelPhelps',
 'The Baltimore Bullet',
 '30 June 1985 in <a href="/place_names/4423">Baltimore, Maryland (USA)</a>',
 '193 cm / 91 kg',
 '6636">NBAC, Baltimore (USA)</a> / 3411">Michigan Wolverines, Ann Arbor (USA)</a>',
 'USA.png" style="padding-right: 2px; vertical-align: middle"/> <a href="/countries/USA">United States</a>']

In [18]:
biodata_values = pd.DataFrame(biodata.findChildren(['td']))
biodata_values

,0
0,[Competed in Olympic Games]
1,[Male]
2,"[Michael Fred•Phelps, II]"
3,[Michael•Phelps]
4,[The Baltimore Bullet]
5,"[30 June 1985 in , [Baltimore, Maryland (USA)]]"
6,[193 cm / 91 kg]
7,"[[NBAC, Baltimore (USA)], / , [Michigan Wolve..."
8,"[[], , [United States]]"


In [20]:
df = pd.DataFrame(columns=bio_column)
a_series = pd.Series(bio_data, index=df.columns)
df = df.append(a_series, ignore_index=True)
df

,Type,Sex,Full name,Used name,Nick/petnames,Born,Measurements,Affiliations,NOC
0,Competed in Olympic Games,Male,"Michael FredPhelps, II",MichaelPhelps,The Baltimore Bullet,"30 June 1985 in <a href=""/place_names/4423"">Ba...",193 cm / 91 kg,"6636"">NBAC, Baltimore (USA)</a> / 3411"">Michig...","USA.png"" style=""padding-right: 2px; vertical-a..."


In [ ]:
def strip_html(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data)